## Aim ##
We want to be sure that using the [majority vote](https://en.wikipedia.org/wiki/Majority_judgment) method gives reasonable granularity of scores.

## Rating ##
Reviewers choose a rating for each submission from:

* 0 (reject)
* 1 (accept)
* 2 (promote)

Assuming at least 10 reviewers this gives 220 possible scores (arrangements of ratings). e.g. if there were 3 reviewers '0-0-0', '1-1-1', '2-2-2' and '1-2-3' are just 4 of the possible 10 scores.

## Majority Vote ##
Majority vote is a system that aims to produce a ['fair'](https://en.wikipedia.org/wiki/Majority_judgment#Satisfied_and_failed_criteria) ranking of candidates, in our case 'canidates' refer to submissions and 'voters' refers to a reviewer.

The algorithm used is:

1. Sort all submissions' ratings (to produce a score)
2. Find the median rating of the score
3. Group submissions by their median rating
4. For each member of a group remove one instance of that group's median rating from the member's score
5. Repeat steps 2-4 until the submissions are sorted or each group is empty

As implemented this results in a tree of ordered dictionaries. The keys for each dictionary are the median rating for the child elements. This means that at each nesting level the keys are: 0, 1, or 2. The nesting continues to a fixed depth based on an argument supplied to the ranking function but it is also bound by the combinatorics of the reviewers & scoring (i.e. the number of ways that a submission can be scored by the reviewers). E.g. with 10 reviewers rating either 0, 1, or 2 there are 220 possible scores.

## Ranking ##
Whilst the leaves of the scoring tree are nominally sorted its for ease of manipulation the tree is flattened. Each leaf is assigned a rank by calculating the value of the path to that leaf. E.g. leaf 0-1-2 is 5 (0x9 + 1x3 + 2).

**NB**: because of the nature of the majority vote system the leaf values are not evenly distributed.


In [1]:
from majority_judgement import (rank_submissions, linearise_submissions, generate_scores,
                            get_floor_median, remove_scoring_element)
from copy import deepcopy

In [ ]:
%matplotlib inline

In [ ]:
import matplotlib.pyplot as plt

In [2]:
def calc(score_set):
    _score_set = sorted(deepcopy(score_set))
    power = 3 ** (len(score_set) - 1)
    res = 0
    while _score_set:
        score = get_floor_median(_score_set)
        res += (score * power)
        power = int(power / 3)
        remove_scoring_element(score, _score_set)
    return res

def alt_rank(scores):
    res = {}
    for s in scores:
        score = calc(s)
        res.setdefault(score, 0)
        res[score] += 1
    return res

In [3]:
scores = generate_scores(90, 8)

In [13]:
%timeit linearise_submissions(rank_submissions(deepcopy(scores)))
tree = rank_submissions(deepcopy(scores))
lin_trees = linearise_submissions(tree)
print(lin_trees)

100 loops, best of 3: 2.52 ms per loop
OrderedDict([(0, 2), (1, 8), (2, 1), (10, 4), (11, 5), (20, 1), (37, 15), (38, 5), (40, 9), (41, 19), (47, 4), (50, 10), (74, 1), (77, 6)])


In [14]:
%timeit alt_rank(scores)
maths = alt_rank(scores)
print(sorted(maths.items(), key=lambda x:x))

100 loops, best of 3: 2.82 ms per loop
[(1, 1), (10, 1), (91, 1), (92, 4), (101, 3), (182, 1), (820, 1), (821, 1), (830, 2), (911, 5), (1640, 1), (3007, 2), (3008, 4), (3017, 9), (3098, 5), (3251, 2), (3260, 6), (3287, 1), (3341, 11), (3368, 7), (3371, 1), (3827, 4), (4070, 2), (4097, 5), (4100, 3), (6014, 1), (6257, 2), (6284, 3), (6287, 1)]


In [ ]:
scores = generate_scores(90, 8)

res = rank_submissions(scores, max_depth=5)
lin_res = linearise_submissions(res)

print('Score Count')
for score, count in lin_res.items():
    print('{0:5d} {1:<3d}'.format(score, count))

print('\nTotal:', sum(lin_res.values()))

In [ ]:
def brute_force_it(n_loops, n_submissions=90, n_reviewers=10, max_depth=10):
    res = {}
    for i in range(n_loops):
        scores = generate_scores(n_submissions, n_reviewers)
        ranked = rank_submissions(scores, max_depth)
        linearised = linearise_submissions(ranked)
        for position, count in linearised.items():
            if position not in res:
                res[position] = []
            res[position].append(count)
    return res

In [ ]:
def analyse_brute(brute_data, percent_accepted=0.2, count_accepted=250):
    # Turn into ordered (score, count) pairs
    score_counts = [(i, sum(j)) for i, j in brute_data.items()]
    score_counts = sorted(score_counts, key=lambda x: x[0], reverse=True)
    total = sum([j for i, j in score_counts])

    # Find scores for top 20%
    target = percent_accepted * total
    running_total = 0
    passing_scores = []

    for score, count in score_counts:
        if running_total + count > target:
            break
        running_total += count
        passing_scores.append(score)
    # Pretty print
    print('Passing scores:', passing_scores)
    print('Account for: {0:2.1f}% of total'.format(100*running_total/total))
    # Make a plot
    x = [i for i, j in score_counts]
    y = [j for i, j in score_counts]
    plt.bar(x,y)

In [ ]:
huge_test_90_initial_subs = brute_force_it(10000)

In [ ]:
analyse_brute(huge_test_90_initial_subs)

In [ ]:
huge_test_400_final_subs = brute_force_it(10000, n_submissions=400)

In [ ]:
# Assume we can accept about 200 things, i.e. 50%
analyse_brute(huge_test_400_final_subs, percent_accepted=0.5)

In [ ]:
from math import factorial

In [ ]:
def nMultichooser(n_reviewers, k_scores):
    """
    Calculates the number of combinations of n_reviewers selecting from 
    k_score options.
    
    e.g. nMultichooser(3,2) gives 6, the number of combinations of 2 values
    including repeats (i.e. 1-1, 1-2, 1-3, 2-2, 2-3, 3-3: 6 combinations)
    """
    n = n_reviewers + k_scores - 1
    numerator = factorial(n)
    denominator = factorial(k_scores)*factorial(n - k_scores)
    return numerator/denominator

In [ ]:
combinatorics = [(i, nMultichooser(i, 3)) for i in range(3, 20)]

print('Reviewers Score range')
for n_reviwers, score in combinatorics:
    print('{0:9d} {1:<4d}'.format(n_reviwers, int(score)))